In [2]:
# Load in data
import pandas as pd
data = pd.read_csv("Zillow_Airbnb.csv")

In [3]:
# View data
data

,Unnamed: 0,RegionID,SizeRank,City,RegionType,StateName,Date,ZHVI,Listings
0,0,394355,29,"Austin, TX",msa,TX,2000-01-31,167608.803584,0
1,1,394355,29,"Austin, TX",msa,TX,2000-02-29,168157.892276,0
2,2,394355,29,"Austin, TX",msa,TX,2000-03-31,168656.684383,0
3,3,394355,29,"Austin, TX",msa,TX,2000-04-30,169450.508196,0
4,4,394355,29,"Austin, TX",msa,TX,2000-05-31,170048.593123,0
...,...,...,...,...,...,...,...,...,...
4721,4721,753899,2,"Los Angeles, CA",msa,CA,2022-10-31,887956.929337,14349
4722,4722,753899,2,"Los Angeles, CA",msa,CA,2022-11-30,884296.450518,13785
4723,4723,753899,2,"Los Angeles, CA",msa,CA,2022-12-31,880012.047421,13042
4724,4724,753899,2,"Los Angeles, CA",msa,CA,2023-01-31,871774.871645,11029


In [4]:
max(data['Listings'])

14807

In [5]:
# Creat groupby object grouped by city
grouped = data.groupby(data['City'])

In [6]:
# Move data into dictionary with city as keys
groupdict = {}
for val in set(data['City'].values):
    groupdict[val] = grouped.get_group(val)

In [7]:
# Create list of first nonzero dates for listings; that is, date of first Airbnb listing
list = []
for key in set(groupdict.keys()):
    list.append(groupdict[key].iloc[groupdict[key]['Listings'].to_numpy().nonzero()[0]].iloc[0]['Date'])


In [8]:
# Convert date values to datetime type (currently strings)
for key in set(groupdict.keys()):
    groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')

/var/folders/bm/jxfvs9l9111c7zs8d_bdq4zh0000gn/T/ipykernel_6354/60717528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')
/var/folders/bm/jxfvs9l9111c7zs8d_bdq4zh0000gn/T/ipykernel_6354/60717528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupdict[key]['Date'] = pd.to_datetime(groupdict[key]['Date'], format='%Y-%m-%d')
/var/folders/bm/jxfvs9l9111c7zs8d_bdq4zh0000gn/T/ipykernel_6354/60717528.py:3: SettingWithCopyWa

In [9]:
# Among all cities, the latest date of first listing is October 2011; as a result, for simplicity sake, we'll start with Dec 31, 2011, equivalent to the start of 2012.
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key][groupdict[key]['Date'] >= '2011-12-31']

In [10]:
# Remove columns not to be included in models
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key].drop(columns = ['Unnamed: 0', 'RegionID', 'SizeRank', 'City', 'RegionType', 'StateName'])

In [11]:
# Load in secondary features (federal funds rate, CPI, unemployment rate)
rate = pd.read_csv("FEDFUNDS.csv")
cpi = pd.read_csv("CPIAUCSL.csv")
unrate = pd.read_csv("UNRATE.csv")

In [12]:
 unrate

,DATE,UNRATE
0,12/31/11,8.3
1,1/31/12,8.3
2,2/29/12,8.2
3,3/31/12,8.2
4,4/30/12,8.2
...,...,...
130,10/31/22,3.6
131,11/30/22,3.5
132,12/31/22,3.4
133,1/31/23,3.6


In [13]:
max(unrate['UNRATE'])

14.7

In [14]:
# Convert date columns to datetime type
rate['DATE'] = pd.to_datetime(rate['DATE'], format='%m/%d/%y')
cpi['DATE'] = pd.to_datetime(cpi['DATE'], format='%m/%d/%y')
unrate['DATE'] = pd.to_datetime(unrate['DATE'], format='%m/%d/%y')

In [15]:
# Join new values into dictionary of dataframes
for key in set(groupdict.keys()):
    groupdict[key] = groupdict[key].merge(rate, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE', 'DATE'])
    groupdict[key] = groupdict[key].merge(cpi, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE'])
    groupdict[key] = groupdict[key].merge(unrate, left_on = 'Date', right_on = 'DATE')
    groupdict[key] = groupdict[key].drop(columns = ['DATE'])

In [16]:
# Create dictionary to use for first difference model
diffdict = {}
for key in set(groupdict.keys()):
    diffdict[key] = groupdict[key].copy()

In [17]:
# Change values to difference between months for diffdict, remove first row (NaN value)
for key in set(diffdict.keys()):
    diffdict[key]['ZHVI'] = diffdict[key]['ZHVI'].diff()
    diffdict[key]['Listings'] = diffdict[key]['Listings'].diff()
    diffdict[key]['FEDFUNDS'] = diffdict[key]['FEDFUNDS'].diff()
    diffdict[key]['CPIAUCSL'] = diffdict[key]['CPIAUCSL'].diff()
    diffdict[key]['UNRATE'] = diffdict[key]['UNRATE'].diff()
    diffdict[key] = diffdict[key][diffdict[key]['Date'] > '2011-12-31']

In [18]:
# Define x's and y's for MLR model
reg_xs = {}
reg_ys = {}
for key in set(groupdict.keys()):
    reg_xs[key] = groupdict[key].drop(columns = ['Date', 'ZHVI'])
    reg_ys[key] = groupdict[key]['ZHVI']

In [19]:
# Define x's and y's for first difference model
diff_xs = {}
diff_ys = {}
for key in set(groupdict.keys()):
    diff_xs[key] = diffdict[key].drop(columns = ['Date', 'ZHVI'])
    diff_ys[key] = diffdict[key]['ZHVI']

In [20]:
# Run MLR model; get p-values, R^2 values, and summary ourputs
import statsmodels.api as sm
reg_models = {}
p_values_reg = {}
R_sq_reg = {}
summaries_reg = {}
for key in set(groupdict.keys()):
    reg_models[key] = sm.OLS(reg_ys[key],reg_xs[key])
    fii = reg_models[key].fit()
    p_values_reg[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_reg[key] = fii.summary2().tables[0][1][6]
    summaries_reg[key] = fii.summary()

In [21]:
# Run first difference model; get p-values, R^2 values, and summary ourputs
import statsmodels.api as sm
diff_models = {}
p_values_diff = {}
R_sq_diff = {}
summaries_diff = {}
for key in set(groupdict.keys()):
    diff_models[key] = sm.OLS(diff_ys[key],diff_xs[key])
    fii = diff_models[key].fit()
    p_values_diff[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_diff[key] = fii.summary2().tables[0][1][6]
    summaries_diff[key] = fii.summary()

In [22]:
# The coefficients aren't highly interpretable for determining variable impact beyond p-values; normalize variables to allow for interpretability
scaled_reg_xs = {}
scaled_reg_ys = {}
scaled_diff_xs = {}
scaled_diff_ys = {}
for key in set(groupdict.keys()):
    scaled_reg_xs[key] = (reg_xs[key]-reg_xs[key].mean())/reg_xs[key].std()
    scaled_reg_ys[key] = (reg_ys[key]-reg_ys[key].mean())/reg_ys[key].std()
    scaled_diff_xs[key] = (diff_xs[key]-diff_xs[key].mean())/diff_xs[key].std()
    scaled_diff_ys[key] = (diff_ys[key]-diff_ys[key].mean())/diff_ys[key].std()

In [23]:
# Run MLR model on scaled data; get p-values, R^2 values, and summary ourputs
reg_models_scaled = {}
p_values_reg_scaled = {}
R_sq_reg_scaled = {}
summaries_reg_scaled = {}
for key in set(groupdict.keys()):
    reg_models_scaled[key] = sm.OLS(scaled_reg_ys[key],scaled_reg_xs[key])
    fii = reg_models_scaled[key].fit()
    p_values_reg_scaled[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_reg_scaled[key] = fii.summary2().tables[0][1][6]
    summaries_reg_scaled[key] = fii.summary()

In [24]:
# Run first difference model on scaled data; get p-values, R^2 values, and summary ourputs
diff_models_scaled = {}
p_values_diff_scaled = {}
R_sq_diff_scaled = {}
summaries_diff_scaled = {}
for key in set(groupdict.keys()):
    diff_models_scaled[key] = sm.OLS(scaled_diff_ys[key],scaled_diff_xs[key])
    fii = diff_models_scaled[key].fit()
    p_values_diff_scaled[key] = fii.summary2().tables[1]['P>|t|']
    R_sq_diff_scaled[key] = fii.summary2().tables[0][1][6]
    summaries_diff_scaled[key] = fii.summary()

In [25]:
summaries_reg_scaled

{'Austin, TX': <class 'statsmodels.iolib.summary.Summary'>
 """
                                  OLS Regression Results                                
 Dep. Variable:                   ZHVI   R-squared (uncentered):                   0.983
 Model:                            OLS   Adj. R-squared (uncentered):              0.983
 Method:                 Least Squares   F-statistic:                              1928.
 Date:                Fri, 21 Apr 2023   Prob (F-statistic):                   2.60e-115
 Time:                        15:20:33   Log-Likelihood:                          85.152
 No. Observations:                 135   AIC:                                     -162.3
 Df Residuals:                     131   BIC:                                     -150.7
 Df Model:                           4                                                  
 Covariance Type:            nonrobust                                                  
                  coef    std err          t  